In [1]:
import pandas as pd
import json
import re
import json
from bs4 import BeautifulSoup
import bs4
import requests
import os
from tqdm import tqdm
import pickle
from law_query_private import LawQuery,Tree
from slugify import slugify
import gzip

In [2]:
# # get luat hien hanh
# req = requests.get('https://thuvienphapluat.vn/chinh-sach-phap-luat-moi/vn/thoi-su-phap-luat/tu-van-phap-luat/42248/danh-muc-luat-bo-luat-hien-hanh-tai-viet-nam')
# soup = BeautifulSoup(req.text, 'html.parser')
# luat_hien_hanh = soup.select('.newcontent a')
# urls = []
# for i in luat_hien_hanh:
#     urls.append(i['href'])
# urls = urls[1:]
# # save to urls.txt
# with open('urls.txt', 'w') as f:
#     f.write('\n'.join(urls))

In [3]:
urls = []
with open('urls2.txt', 'r') as f:
    urls = f.read().split('\n')

In [4]:
def get_text(url):
    # get metadata
    metadata_url='https://thuvienphapluat.vn/AjaxLoadData/LoadLuocDo.aspx?LawID={}&IstraiNghiem=True'
    id = url.replace('.aspx','').split('-')[-1]
    req=requests.get(metadata_url.format(id),headers={'referer': url})
    if req.status_code != 200:
        raise Exception('Error when get metadata')
    soup = BeautifulSoup(req.text, 'html.parser')
    metadata = soup.select_one('#viewingDocument')
    metadata = {
        'ten_van_ban': metadata.select_one('#viewingDocument > div:nth-child(1)').getText(strip=True), 
        'so_hieu_van_ban': metadata.select_one('#viewingDocument > div:nth-child(2) > div.ds.fl').getText(strip=True),
        'loai_van_ban': metadata.select_one('#viewingDocument > div:nth-child(3) > div.ds.fl').getText(strip=True),
        'linhvuc': metadata.select_one('#viewingDocument > div:nth-child(4) > div.ds.fl').getText(strip=True),
        'noi_ban_hanh': metadata.select_one('#viewingDocument > div:nth-child(5) > div.ds.fl').getText(strip=True),
        'nguoi_ky': metadata.select_one('#viewingDocument > div:nth-child(6) > div.ds.fl').getText(strip=True),
        'ngay_ban_hanh': metadata.select_one('#viewingDocument > div:nth-child(7) > div.ds.fl').getText(strip=True),
        'ngay_hieu_luc': metadata.select_one('#viewingDocument > div:nth-child(8) > div.ds.fl').getText(strip=True),
        'ngay_cong_bao': metadata.select_one('#viewingDocument > div:nth-child(9) > div.ds.fl').getText(strip=True),
        'so_cong_bao': metadata.select_one('#viewingDocument > div:nth-child(10) > div.ds.fl').getText(strip=True),
        'tinh_trang': metadata.select_one('#viewingDocument > div:nth-child(11) > div.ds.fl').getText(strip=True),
    }
    req=requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    html = soup.find('div', {'class':'content1'})
    for element in html(text=lambda text: isinstance(text, bs4.Comment)):
        element.extract()
    for a in html.find_all('a', href=lambda x: x and x[0] == '#'):
        a.extract()
    for s in html.find_all(['script','style']):
        s.extract()
    ps = html.find_all(['p','h1','h2','h3','h4','h5','h6'])
    content = ''
    for p in ps:
        text = p.text
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        content += text + '\n'
    # delete - more than 3
    content = re.sub(r'-{3,}', '', content)
    content = re.sub(r'\*{3,}', '', content)
    # remove line with empty content
    content = re.sub(r'^\s+$', '', content, flags=re.MULTILINE)
    # name = soup.find('h1').text.strip()
    matches = re.finditer(r'^(chương [\d\w]+.*|phần thứ [\d\w]+.*)$', content, re.MULTILINE | re.IGNORECASE)
    for i,match in enumerate(matches):
        end = match.end()
        while end < len(content) and content[end] != '\n':
            end += 1
        if end < len(content) and content[end] == '\n':
            content = content[:end] + ':' + content[end+1:]
    return metadata,content,html.prettify()

In [15]:
items = []
for url in urls:
    try:
        metadata,text,html = get_text(url)
        print(metadata['ten_van_ban'])
        name_slug = slugify(metadata['ten_van_ban'])[:50]
        items.append({'metadata':metadata,'path':name_slug})
        folder_path = os.path.join('documents', name_slug)
        os.makedirs(folder_path, exist_ok=True)
        with open(os.path.join(folder_path, 'content.txt'), 'w', encoding='utf-8') as f:
            f.write(text)
        with open(os.path.join(folder_path, 'content.html'), 'w', encoding='utf-8') as f:
            f.write(html)
    except Exception as e:
        print(e)
        print(url)
        continue
df = pd.DataFrame(items)
# expose metadata
df = pd.concat([df, df['metadata'].apply(pd.Series)], axis=1)
df['metadata'] = df['metadata'].apply(lambda x: json.dumps(x, ensure_ascii=False))
df.to_csv('documents/data.csv', index=False)

Luật Bảo hiểm xã hội 2014
Văn bản hợp nhất 2089/VBHN-BHXH năm 2020 hợp nhất Quyết định về Quy trình thu bảo hiểm xã hội, bảo hiểm y tế, bảo hiểm thất nghiệp, bảo hiểm tai nạn lao động, bệnh nghề nghiệp; quản lý sổ bảo hiểm xã hội, thẻ bảo hiểm y tế do Bảo hiểm xã hội Việt Nam ban hành
Thông tư 59/2015/TT-BLĐTBXH quy định chi tiết và hướng dẫn thi hành một số điều của Luật bảo hiểm xã hội về bảo hiểm xã hội bắt buộc do Bộ trưởng Bộ Lao động - Thương binh và Xã hội ban hành
Nghị định 115/2015/NĐ-CP hướng dẫn Luật bảo hiểm xã hội về bảo hiểm xã hội bắt buộc
Nghị định 146/2018/NĐ-CP hướng dẫn Luật bảo hiểm y tế
Nghị định 28/2015/NĐ-CP hướng dẫn Luật Việc làm về bảo hiểm thất nghiệp
Luật việc làm 2013
Bộ luật Lao động 2019
Thông tư 56/2017/TT-BYT về hướng dẫn Luật bảo hiểm xã hội và Luật an toàn vệ sinh lao động thuộc lĩnh vực y tế do Bộ trưởng Bộ Y tế ban hành
Nghị định 134/2015/NĐ-CP hướng dẫn Luật Bảo hiểm xã hội về bảo hiểm xã hội tự nguyện
Quyết định 28/2021/QĐ-TTg quy định về thực hiệ

In [16]:
def make_content_for_tree(text):
    re_footer = r'^(Bộ luật này đã được Quốc hội .*|Luật này được Quốc hội.*|Bộ luật này được Quốc hội .*|Luật này đã được Quốc hội .*)'
    matches = re.split(re_footer, text, flags=re.MULTILINE)
    text = matches[0]
    return text

In [17]:
CREATE_CONTENT_TREE=True
df = pd.read_csv('documents/data.csv')
for row in df.itertuples():
    print(row.path)
    folder_path = os.path.join('documents', row.path)
    text = open(os.path.join(folder_path, 'content.txt'), encoding='utf-8').read()
    raw_text = text
    text = make_content_for_tree(text)
    if CREATE_CONTENT_TREE:
        with open(os.path.join(folder_path, 'content_tree.txt'), 'w', encoding='utf-8') as f:
            f.write(text)
    text = open(os.path.join(folder_path, 'content_tree.txt'), encoding='utf-8').read()
    tree = Tree(metadata= json.loads(row.metadata),content=text,raw_text=raw_text)
    with open(os.path.join(folder_path, 'debug_tree.txt'), 'w', encoding='utf-8') as f:
        f.write(str(tree))
    with open(os.path.join(folder_path, 'tree.pkl'), 'wb') as f:
        pickle.dump(tree, f)
    with open(os.path.join(folder_path, 'tree.json'), 'w', encoding='utf-8') as f:
        f.write(json.dumps(tree.export(), indent=4, ensure_ascii=False))
    with open(os.path.join(folder_path, 'tree.json.gz'), 'wb') as f:
        f.write(gzip.compress(json.dumps(tree.export(), ensure_ascii=False).encode('utf-8')))
    # with open('documents/{}.json.gz'.format(row.path), 'wb') as f:
        # f.write(gzip.compress(json.dumps(tree.export(), ensure_ascii=False).encode('utf-8')))

luat-bao-hiem-xa-hoi-2014
van-ban-hop-nhat-2089-vbhn-bhxh-nam-2020-hop-nhat-
thong-tu-59-2015-tt-bldtbxh-quy-dinh-chi-tiet-va-h
nghi-dinh-115-2015-nd-cp-huong-dan-luat-bao-hiem-x
nghi-dinh-146-2018-nd-cp-huong-dan-luat-bao-hiem-y
nghi-dinh-28-2015-nd-cp-huong-dan-luat-viec-lam-ve
luat-viec-lam-2013
bo-luat-lao-dong-2019
thong-tu-56-2017-tt-byt-ve-huong-dan-luat-bao-hiem
nghi-dinh-134-2015-nd-cp-huong-dan-luat-bao-hiem-x
quyet-dinh-28-2021-qd-ttg-quy-dinh-ve-thuc-hien-ch
luat-bao-hiem-y-te-2008
luat-bao-hiem-y-te-sua-doi-2014
quyet-dinh-166-qd-bhxh-nam-2019-ve-quy-trinh-giai-
nghi-dinh-61-2020-nd-cp-ve-sua-doi-nghi-dinh-28-20
